# Tile masking
Masking is a common operation to use when performing analysis across tiles. Tile masking allows one to compare two tiles, a value tile and a masking tile. The masking tile is often a binary tile, and can be thought of as being placed on top of the other tile. Masking cells set to 0 are opaque, and cells that are 1 are transparent. In this way, value cells that correspond to masking cells that are 0 become nullified and value cells corresponding to 1 are untouched.

Initial configs

In [9]:
import astraea.spark.rasterframes._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import org.apache.spark.sql._
import geotrellis.raster.{Tile, NODATA}
// import geotrellis.raster.summary._
import geotrellis.raster.render._

implicit val spark = SparkSession.builder().
  master("local").appName("RasterFrames").
  config("spark.ui.enabled", "false").
  getOrCreate().
  withRasterFrames

def readTiff(name: String): SinglebandGeoTiff = SinglebandGeoTiff(s"../samples/$name")

import astraea.spark.rasterframes._
import geotrellis.raster.io.geotiff.SinglebandGeoTiff
import org.apache.spark.sql._
import geotrellis.raster.{Tile, NODATA}
import geotrellis.raster.render._
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@4d8d925d
readTiff: (name: String)geotrellis.raster.io.geotiff.SinglebandGeoTiff


In [10]:
val filenamePattern = "L8-B%d-Elkton-VA.tiff"
val bandNumbers = 1 to 4
val bandColNames = bandNumbers.map(b ⇒ s"band_$b").toArray

filenamePattern: String = L8-B%d-Elkton-VA.tiff
bandNumbers: scala.collection.immutable.Range.Inclusive = Range(1, 2, 3, 4)
bandColNames: Array[String] = Array(band_1, band_2, band_3, band_4)


In [11]:
val joinedRF = bandNumbers.
  map { b ⇒ (b, filenamePattern.format(b)) }.
  map { case (b, f) ⇒ (b, readTiff(f)) }.
  map { case (b, t) ⇒ t.projectedRaster.toRF(s"band_$b") }.
  reduce(_ spatialJoin _)

joinedRF: astraea.spark.rasterframes.RasterFrame = [spatial_key: struct<col: int, row: int>, band_1: rf_tile ... 3 more fields]


First, we create a masking tile based on a previous one. This tile is full of either 1's or 0's corresponsing to whether the value in that cell is greater than a threshold. 

In [17]:
val threshold = udf((t: Tile) => {t.map(x => if (false) x else NODATA)})
threshold(joinedRF("band_1"))

threshold: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,org.apache.spark.sql.gt.types.TileUDT@12598ca2,Some(List(org.apache.spark.sql.gt.types.TileUDT@12598ca2)))
res7: org.apache.spark.sql.Column = UDF(band_1)


In [18]:
val withMaskedTile = joinedRF.withColumn("maskTile", threshold($"band_1")).asRF
withMaskedTile.select(noDataCells(threshold(joinedRF("band_1"))), noDataCells($"maskTile")).show()

+---------------------+---------------------+
|noDataCells(scalaudf)|noDataCells(maskTile)|
+---------------------+---------------------+
|                    0|                    0|
+---------------------+---------------------+



withMaskedTile: astraea.spark.rasterframes.RasterFrame = [spatial_key: struct<col: int, row: int>, band_1: rf_tile ... 4 more fields]


Mask is an operation that takes two tiles and eliminates data from one cell that corresponds to a value in the other. For instance, if a cell in the masking tile contains a 0, the corresponding cell in the other tile would be set to NoData. If the masking cell contained a 1, the corresponding cell would have had its value untouched.

In [ ]:
val masked = withMaskedTile.withColumn("masked", mask($"band_2", $"maskTile")).asRF

In [ ]:
val maskRaster = masked.toRaster($"masked", 466, 428)
val b2 = masked.toRaster($"band_2", 466, 428)

val brownToGreen = ColorRamp(
  RGBA(166,97,26,255),
  RGBA(223,194,125,255),
  RGBA(245,245,245,255),
  RGBA(128,205,193,255),
  RGBA(1,133,113,255)
).stops(128)

val colors = ColorMap.fromQuantileBreaks(maskRaster.tile.histogramDouble(), brownToGreen)

maskRaster.tile.color(colors).renderPng().write("mask.png")
b2.tile.color(colors).renderPng().write("b2.png")

In [ ]:
masked.select(tileStats($"masked")).show()
masked.select(tileStats($"band_2")).show()